## Homework
This time we employ the cross validation to figure out the best model for spam filter.

1. Use the function **GridSearchCV** to find out the best combination of parameter for logistic regression.( Set *cv=5* and *scoring = 'accuracy'*). 

In [74]:
import pandas as pd
import numpy as np

spam_train = pd.read_csv('data/spam_train.csv')
x_train = np.array(spam_train.iloc[:, :57])
y_train = np.array(spam_train.iloc[:, -1])


spam_test = pd.read_csv('data/spam_test.csv')
x_test = np.array(spam_test.iloc[:, :57])
y_test = np.array(spam_test.iloc[:, -1])

In [78]:
import sklearn.cross_validation as cv
import sklearn.grid_search as gs
from sklearn import linear_model
para_grid = [{'penalty': ['l1', 'l2'], 'fit_intercept': [False, True], 
              'C':np.logspace(-5, 5, 100)}]
logit = linear_model.LogisticRegression()
para_search = gs.GridSearchCV(logit, para_grid, cv = 5, scoring='accuracy')
para_search.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'penalty': ['l1', 'l2'], 'C': array([  1.00000e-05,   1.26186e-05, ...,   7.92483e+04,   1.00000e+05]), 'fit_intercept': [False, True]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

    - What's the best combination?
    - What's the best score?
    - Refit the best estimator on the whole data set. How many coefficients are shrinked to 0?(Hint: the absolute value of coefficients that are smaller than 1e-4.) 
    - What's the corresponding training error and test error? (Training error is the model performance on spam_train, while test error is the performance on spam_test.)

In [85]:
### your solution
# 1.
print para_search.best_params_

# 2.
print para_search.best_score_

# 3.
logit_best = para_search.best_estimator_
print np.sum(np.abs(logit_best.coef_) < 1e-4)
# 4.
print "Training error: %.5f" % (1-logit_best.score(x_train, y_train))
print "Training error: %.5f" % (1-logit_best.score(x_test, y_test))

# 5.
coef_accuracy = logit_best.coef_

{'penalty': 'l2', 'C': 4862.6015800653531, 'fit_intercept': True}
0.929565217391
0
Training error: 0.06261
Training error: 0.07214


2.Set *scoring = 'roc_auc'* and search again, what's the best parameters? Fit the best estimator on the spam_train data set. What's the training error and test error?

In [14]:
### your solution
y_train = [1 if i == 'spam' else 0 for i in y_train]
y_test = [1 if i == 'spam' else 0 for i in y_test]

para_search = gs.GridSearchCV(logit, para_grid, cv = 5, scoring='roc_auc')

para_search.fit(x_train, y_train) 
logit_best = para_search.best_estimator_

LogisticRegression(C=1.4174741629268048, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [15]:
para_search.best_params_

{'C': 1.4174741629268048, 'fit_intercept': True, 'penalty': 'l1'}

In [16]:
print "Training error: %.5f" %logit_best.score(x_train, y_train)
print "Training error: %.5f" %logit_best.score(x_test, y_test)

Training error: 0.93609
Training error: 0.92829


3.In this exercise, we will predict the number of applications received(*Apps*) using the other variables in the College data set.

The features and the target variable are prepared as $x$ and $y$.

In [49]:
import pandas as pd
college = pd.read_csv('data/college.csv')
x = college.iloc[:, 2:]
y = college.iloc[:, 1]
print college.shape
college.head()

(777, 18)


,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


- (1) Split this data into a training set and a test set with train_size=0.5.(Hint: Use the function **sklearn.cross_validation.train_test_split** , set *random_state=0* and *tran_size=0.5*.)

- (2) Fit a linear model on the training set and report the training error and test error(mean squared error, you can use the function *sklearn.metrics.mean_squared_error*).

- (3) Fit a ridge regression on the training set, with $\alpha$ chosen by the cross validation. Report the training error and test error.

- (4) Fit a lasso on the training set, with $\alpha$ chosen by the cross validation. Report the training error and test error

- (5) Compare the results obtained, what do you find?

In [50]:
### your solution
import sklearn.cross_validation as cv
x_train, x_test, y_train, y_test = cv.train_test_split(x, y, train_size = 0.5, random_state=0)
# (2)
import sklearn.linear_model as lm
from sklearn.metrics import *
ols = lm.LinearRegression().fit(x_train, y_train)
print "Training error of linear model is: %.2f" %mean_squared_error(ols.predict(x_train), y_train) + 5*' ' + "Test error of linear model is: %.2f" %mean_squared_error(ols.predict(x_test), y_test)

Training error of linear model is: 1113145.99     Test error of linear model is: 1244800.27


In [57]:
# (3)
ridgeCV = lm.RidgeCV(np.logspace(-5,5, 100), cv=5)
ridge = ridgeCV.fit(x_train, y_train)
print "Training error of ridge  model is: %.2f" %mean_squared_error(ridge.predict(x_train), y_train) + 5*' ' + "Test error of ridge  model is: %.2f" %mean_squared_error(ridge.predict(x_test), y_test)

Training error of ridge  model is: 1113772.12     Test error of ridge  model is: 1240199.69


In [60]:
# (4)
lassoCV = lm.LassoCV(alphas= np.logspace(-5,5, 100), cv=5)
lasso = lassoCV.fit(x_train, y_train)
print "Training error of lasso  model is: %.2f" %mean_squared_error(lasso.predict(x_train), y_train) + 5*' ' + "Test error of lasso  model is: %.2f" %mean_squared_error(lasso.predict(x_test), y_test)

Training error of lasso  model is: 1114287.76     Test error of lasso  model is: 1235860.02


(5)
- Ridge and Lasso performs worse than linear model on the training set because of the regularization.
- But ridge and lasso predict better on the test set.
- Lasso performs worst on the training set but performs best on the test set.
- It's dangerous to use training error estimate test error.

(Optional) 4.This time  we will try to predict the variable *Private* using the other variables in the College data set. The features and target variable are prepared for you.

In [67]:
x = college.iloc[:, 1:]
y = college.iloc[:, 0]

- (1) Split this data into a training set and a test set with train_size=0.5(Hint: Use the function **sklearn.cross_validation.train_test_split** , set *random_state=1* and *tran_size=0.5*.)]

- (2) Fit a logistic regression with regularizaton. Use the function **GridSearchCV** to fint out the best parameters.

In [68]:
grid_para_logit = [{'penality': ['l1', 'l2'], 'alpha': np.logspace(-5, 5, 100)}]

    - What's the best parameters?
    - Refit the model on the training set with best parameters. What's the training error and test error?
    
- (3) Fit a KNN model. Use the function **GridSearchCV** to fint out the appropriate parameter *n_neighbors*. Refit the model on the training set and report the training error and test error.

- (4) Compare the results of logistic regression and KNN.

In [69]:
### your solution

## (1)
x_train, x_test, y_train, y_test = cv.train_test_split(x, y, train_size=0.5, random_state=1)

## (2)
import sklearn.grid_search as gs
logit = lm.LogisticRegression()
grid_para_logit = [{'penalty': ['l1', 'l2'], 'C':np.logspace(-5, 5, 100)}]
grid_search_logit = gs.GridSearchCV(logit, grid_para_logit,scoring = 'accuracy') ## search
grid_search_logit.fit(x_train, y_train)
logit = grid_search_logit.best_estimator_

print "The most appropriate parameter are: " + str(grid_search_logit.best_params_)
print "The training error of logistic: %.5f" %(1-logit.score(x_train, y_train))
print "The test     error of logistic: %.5f" %(1-logit.score(x_test, y_test))


The most appropriate parameter are: {'penalty': 'l1', 'C': 0.027185882427329403}
The training error of logistic: 0.04381
The test     error of logistic: 0.05913


In [70]:
## (3)
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier()
grid_para_knn = [{'n_neighbors': range(3, 31)}]
grid_search_knn = gs.GridSearchCV(knn, grid_para_knn, scoring='accuracy').fit(x_train, y_train)
print grid_search_knn.best_params_
knn = grid_search_knn.best_estimator_
print "The training error of KNN: %.5f" %(1- knn.score(x_train, y_train))
print "The test     error of KNN: %.5f" %(1- knn.score(x_test, y_test))

{'n_neighbors': 26}
The training error of KNN: 0.06186
The test     error of KNN: 0.07198


(4)
+ The test error is slightly higher than training error both in logistic regression and KNN.
+ Logistic regression is slightly better than KNN in this case.